# Minimal Ride Hailing Example

![Feast Data Flow](./images/data-flow.png)

## Introduction

For this quick start, we will:
1. Register two driver features, one for driver statistics, the other for driver trips. Driver statistics are updated on daily basis, whereas driver trips are updated in real time.
2. Creates a driver dataset, then use Feast SDK to retrieve the features corresponding to these drivers from an offline store.
3. Store the features in an online store (Redis), and retrieve the features via Feast SDK.

## Features Registry (Feast Core)

### Configuration

In [1]:
!pip install protobuf gcsfs feast -U -q --user

Configurations can be provided in three different methods:

In [2]:
import os
import time
from pprint import pprint
from datetime import datetime
now = datetime.now() # current date and time
date_time_str = now.strftime("%m-%d-%Y-%H-%M-%S")
print(date_time_str)

03-25-2021-13-44-40


If you are following the quick start guide, all required configurations to follow the remainder of the tutorial should have been setup, in the form of environmental variables, as showned below. The configuration values may differ depending on the environment. For a full list of configurable values and explanation, please refer to the user guide.

### Basic Imports and Feast Client initialization

In [3]:
from feast import Client, Feature, Entity, ValueType, FeatureTable
from feast.data_source import FileSource, KafkaSource
from feast.data_format import ParquetFormat, AvroFormat
#client = Client()
staging_bucket = 'gs://my-feast-playground-s-11-35de9347/'
client = Client(
    core_url="feast-release-feast-core:6565",
    serving_url="feast-release-feast-serving:6566",
    spark_launcher="k8s",
    spark_staging_location=staging_bucket,
    spark_k8s_namespace="default",
    executor_instances=2,
    redis_host="feast-release-redis-headless",
    historical_feature_output_location=f"{staging_bucket}historical",
)
client.set_project("default")
staging_bucket = 'gs://my-feast-playground-s-11-35de9347/'
os.environ['FEAST_SPARK_EXTRA_OPTIONS'] = '--jars https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar --conf spark.hadoop.fs.gs.impl=com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem'
pprint({key: value for key, value in os.environ.items() if key.startswith("FEAST_")})



{'FEAST_CORE_URL': 'feast-release-feast-core:6565',
 'FEAST_HISTORICAL_FEATURE_OUTPUT_LOCATION': 'file:///home/jovyan/historical_feature_output',
 'FEAST_HISTORICAL_SERVING_URL': 'feast-release-feast-batch-serving:6566',
 'FEAST_JUPYTER_SERVICE_PORT': 'tcp://10.79.252.133:80',
 'FEAST_JUPYTER_SERVICE_PORT_80_TCP': 'tcp://10.79.252.133:80',
 'FEAST_JUPYTER_SERVICE_PORT_80_TCP_ADDR': '10.79.252.133',
 'FEAST_JUPYTER_SERVICE_PORT_80_TCP_PORT': '80',
 'FEAST_JUPYTER_SERVICE_PORT_80_TCP_PROTO': 'tcp',
 'FEAST_JUPYTER_SERVICE_SERVICE_HOST': '10.79.252.133',
 'FEAST_JUPYTER_SERVICE_SERVICE_PORT': '80',
 'FEAST_REDIS_HOST': 'feast-release-redis-master',
 'FEAST_RELEASE_FEAST_CORE_PORT': 'tcp://10.79.242.226:80',
 'FEAST_RELEASE_FEAST_CORE_PORT_6565_TCP': 'tcp://10.79.242.226:6565',
 'FEAST_RELEASE_FEAST_CORE_PORT_6565_TCP_ADDR': '10.79.242.226',
 'FEAST_RELEASE_FEAST_CORE_PORT_6565_TCP_PORT': '6565',
 'FEAST_RELEASE_FEAST_CORE_PORT_6565_TCP_PROTO': 'tcp',
 'FEAST_RELEASE_FEAST_CORE_PORT_80_TCP

### Declare Features and Entities

Entity defines the primary key(s) associated with one or more feature tables. The entity must be registered before declaring the associated feature tables. 

In [4]:
driver_id = Entity(name="driver_id", description="Driver identifier", value_type=ValueType.INT64)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# Daily updated features 
acc_rate = Feature("acc_rate", ValueType.FLOAT)
conv_rate = Feature("conv_rate", ValueType.FLOAT)
avg_daily_trips = Feature("avg_daily_trips", ValueType.INT32)

# Real-time updated features
trips_today = Feature("trips_today", ValueType.INT32)

```python
FeatureTable(
    name = "driver_statistics",
    entities = ["driver_id"],
    features = [
        acc_rate,
        conv_rate,
        avg_daily_trips
    ]
    ...
)
```


```python
FeatureTable(
    name = "driver_trips",
    entities = ["driver_id"],
    features = [
        trips_today
    ]
    ...
)

```

![Features Join](./images/features-join.png)

```python
FeatureTable(
    ...,
    batch_source=FileSource(  # Required
        file_format=ParquetFormat(),
        file_url="gs://feast-demo-data-lake",
        ...
    ),
    stream_source=KafkaSource(  # Optional
        bootstrap_servers="...",
        topic="driver_trips",
        ...
    )
```

Feature tables group the features together and describe how they can be retrieved. The following examples assume that the feature tables are stored on the local file system, and is accessible from the Spark cluster. If you have setup a GCP service account, you may use GCS instead as the file source.

`batch_source` defines where the historical features are stored. It is also possible to have an optional `stream_source`, which the feature values are delivered continuously.

For now we will define only `batch_source` for both `driver_statistics` and `driver_trips`, and demonstrate the usage of `stream_source` in later part of the tutorial.

In [6]:
# This is the location we're using for the offline feature store.
import os
# Offline data will be stored in this location
demo_data_location = staging_bucket+date_time_str
print(demo_data_location)

gs://my-feast-playground-s-11-35de9347/03-25-2021-13-44-40


In [7]:
driver_statistics_source_uri = os.path.join(demo_data_location, "driver_statistics")

driver_statistics = FeatureTable(
    name = "driver_statistics",
    entities = ["driver_id"],
    features = [
        acc_rate,
        conv_rate,
        avg_daily_trips
    ],
    batch_source=FileSource(
        event_timestamp_column="datetime",
        created_timestamp_column="created",
        file_format=ParquetFormat(),
        file_url=driver_statistics_source_uri,
        date_partition_column="date"
    )
)

In [8]:
driver_trips_source_uri = os.path.join(demo_data_location, "driver_trips")


driver_trips = FeatureTable(
    name = "driver_trips",
    entities = ["driver_id"],
    features = [
        trips_today
    ],
    batch_source=FileSource(
        event_timestamp_column="datetime",
        created_timestamp_column="created",
        file_format=ParquetFormat(),
        file_url=driver_trips_source_uri,
        date_partition_column="date"
    )
)

### Registering entities and feature tables in Feast Core

In [9]:
client.apply(driver_id)
client.apply(driver_statistics)
client.apply(driver_trips)

In [10]:
print(client.get_feature_table("driver_statistics").to_yaml())
print(client.get_feature_table("driver_trips").to_yaml())

spec:
  name: driver_statistics
  entities:
  - driver_id
  features:
  - name: avg_daily_trips
    valueType: INT32
  - name: conv_rate
    valueType: FLOAT
  - name: acc_rate
    valueType: FLOAT
  batchSource:
    type: BATCH_FILE
    eventTimestampColumn: datetime
    datePartitionColumn: date
    createdTimestampColumn: created
    fileOptions:
      fileFormat:
        parquetFormat: {}
      fileUrl: gs://my-feast-playground-s-11-35de9347/03-25-2021-13-44-40/driver_statistics
meta:
  createdTimestamp: '2021-03-25T12:53:30Z'

spec:
  name: driver_trips
  entities:
  - driver_id
  features:
  - name: trips_today
    valueType: INT32
  batchSource:
    type: BATCH_FILE
    eventTimestampColumn: datetime
    datePartitionColumn: date
    createdTimestampColumn: created
    fileOptions:
      fileFormat:
        parquetFormat: {}
      fileUrl: gs://my-feast-playground-s-11-35de9347/03-25-2021-13-44-40/driver_trips
meta:
  createdTimestamp: '2021-03-25T12:53:31Z'



### Populating batch source

Feast is agnostic to how the batch source is populated, as long as it complies to the Feature Table specification. Therefore, any existing ETL tools can be used for the purpose of data ingestion. Alternatively, you can also use Feast SDK to ingest a Panda Dataframe to the batch source.

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime

In [12]:
def generate_entities():
    return np.random.choice(999999, size=100, replace=False)

In [13]:
def generate_trips(entities):
    df = pd.DataFrame(columns=["driver_id", "trips_today", "datetime", "created"])
    df['driver_id'] = entities
    df['trips_today'] = np.random.randint(0, 1000, size=100).astype(np.int32)
    df['datetime'] = pd.to_datetime(
            np.random.randint(
                datetime(2020, 10, 10).timestamp(),
                datetime(2020, 10, 20).timestamp(),
                size=100),
        unit="s"
    )
    df['created'] = pd.to_datetime(datetime.now())
    return df
    

In [14]:
def generate_stats(entities):
    df = pd.DataFrame(columns=["driver_id", "conv_rate", "acc_rate", "avg_daily_trips", "datetime", "created"])
    df['driver_id'] = entities
    df['conv_rate'] = np.random.random(size=100).astype(np.float32)
    df['acc_rate'] = np.random.random(size=100).astype(np.float32)
    df['avg_daily_trips'] = np.random.randint(0, 1000, size=100).astype(np.int32)
    df['datetime'] = pd.to_datetime(
            np.random.randint(
                datetime(2020, 10, 10).timestamp(),
                datetime(2020, 10, 20).timestamp(),
                size=100),
        unit="s"
    )
    df['created'] = pd.to_datetime(datetime.now())
    return df

In [15]:
entities = generate_entities()
stats_df = generate_stats(entities)
trips_df = generate_trips(entities)
time.sleep(10)

In [16]:
client.ingest(driver_statistics, stats_df)
client.ingest(driver_trips, trips_df)
time.sleep(10)

Removing temporary file(s)...
Data has been successfully ingested into FeatureTable batch source.
Removing temporary file(s)...
Data has been successfully ingested into FeatureTable batch source.


## Historical Retrieval For Training

### Point-in-time correction

![Point In Time](./images/pit-2.png)

Feast joins the features to the entities based on the following conditions:
1. Entity primary key(s) value matches.
2. Feature event timestamp is the closest match possible to the entity event timestamp,
   but must not be more recent than the entity event timestamp, and the difference must
   not be greater than the maximum age specified in the feature table, unless the maximum age is not specified.
3. If more than one feature table rows satisfy condition 1 and 2, feature row with the
   most recent created timestamp will be chosen.
4. If none of the above conditions are satisfied, the feature rows will have null values.

In [17]:
import gcsfs
from pyarrow.parquet import ParquetDataset
from urllib.parse import urlparse

In [18]:
def read_parquet(uri):
    parsed_uri = urlparse(uri)
    if parsed_uri.scheme == "file":
        return pd.read_parquet(parsed_uri.path)
    elif parsed_uri.scheme == "gs":
        fs = gcsfs.GCSFileSystem()
        files = ["gs://" + path for path in fs.glob(uri + '/part-*')]
        ds = ParquetDataset(files, filesystem=fs)
        return ds.read().to_pandas()
    elif parsed_uri.scheme == 's3':
        import s3fs
        fs = s3fs.S3FileSystem()
        files = ["s3://" + path for path in fs.glob(uri + '/part-*')]
        ds = ParquetDataset(files, filesystem=fs)
        return ds.read().to_pandas()
    elif parsed_uri.scheme == 'wasbs':
        import adlfs
        fs = adlfs.AzureBlobFileSystem(
            account_name=os.getenv('FEAST_AZURE_BLOB_ACCOUNT_NAME'), account_key=os.getenv('FEAST_AZURE_BLOB_ACCOUNT_ACCESS_KEY')
        )
        uripath = parsed_uri.username + parsed_uri.path
        files = fs.glob(uripath + '/part-*')
        ds = ParquetDataset(files, filesystem=fs)
        return ds.read().to_pandas()
    else:
        raise ValueError(f"Unsupported URL scheme {uri}")

In [19]:
time.sleep(10)
entities_with_timestamp = pd.DataFrame(columns=['driver_id', 'event_timestamp'])
entities_with_timestamp['driver_id'] = np.random.choice(entities, 10, replace=False)
entities_with_timestamp['event_timestamp'] = pd.to_datetime(np.random.randint(
    datetime(2020, 10, 18).timestamp(),
    datetime(2020, 10, 20).timestamp(),
    size=10), unit='s')
entities_with_timestamp

,driver_id,event_timestamp
0,245471,2020-10-18 18:17:51
1,916465,2020-10-19 19:02:17
2,339394,2020-10-18 06:00:49
3,580211,2020-10-18 13:17:49
4,365584,2020-10-18 12:33:22
5,811187,2020-10-19 12:27:17
6,371628,2020-10-18 13:11:42
7,953791,2020-10-18 06:04:12
8,871276,2020-10-19 09:06:34
9,455399,2020-10-18 16:03:02


In [42]:
# get_historical_features will return immediately once the Spark job has been submitted succesfully.
job = client.get_historical_features(
    feature_refs=[
        "driver_statistics:avg_daily_trips",
        "driver_statistics:conv_rate",
        "driver_statistics:acc_rate",
        "driver_trips:trips_today"
    ], 
    entity_source=entities_with_timestamp
)

In [43]:
# get_output_file_uri will block until the Spark job is completed.
time.sleep(10)
output_file_uri = job.get_output_file_uri()
print(output_file_uri)

gs://my-feast-playground-s-11-35de9347/historical/c6c5b16c-e474-416e-b41d-b816a0c2eca8


In [44]:
read_parquet(output_file_uri)

,driver_id,event_timestamp,driver_statistics__avg_daily_trips,driver_statistics__conv_rate,driver_statistics__acc_rate,driver_trips__trips_today
0,371628,2020-10-18 13:11:42,514,0.625232,0.156653,508
1,455399,2020-10-18 16:03:02,859,0.168382,0.101975,338
2,365584,2020-10-18 12:33:22,377,0.339719,0.319552,539
3,811187,2020-10-19 12:27:17,562,0.411371,0.959021,458
4,871276,2020-10-19 09:06:34,459,0.018185,0.031515,828
5,580211,2020-10-18 13:17:49,208,0.483647,0.078080,962
6,916465,2020-10-19 19:02:17,458,0.258407,0.351928,597
7,953791,2020-10-18 06:04:12,347,0.612791,0.075258,731
8,339394,2020-10-18 06:00:49,553,0.341045,0.568152,510
9,245471,2020-10-18 18:17:51,211,0.656132,0.850755,772


The retrieved result can now be used for model training.

## Populating Online Storage with Batch Ingestion

In order to populate the online storage, we can use Feast SDK to start a Spark batch job which will extract the features from the batch source, then load the features to an online store.

In [45]:
time.sleep(10)
job = client.start_offline_to_online_ingestion(
    driver_statistics,
    datetime(2020, 10, 10),
    datetime(2020, 10, 20)
)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
# It will take some time before the Spark Job is completed
time.sleep(10)
job.get_status()
time.sleep(30)

Once the job is completed, the SDK can be used to retrieve the result from the online store.

In [47]:
time.sleep(15)
entities_sample = np.random.choice(entities, 10, replace=False)
entities_sample = [{"driver_id": e} for e in entities_sample]
entities_sample
time.sleep(15)

In [49]:
time.sleep(15)
from feast import Client, Feature, Entity, ValueType, FeatureTable
features = client.get_online_features(
    feature_refs=["driver_statistics:avg_daily_trips"],
    entity_rows=entities_sample).to_dict()
features

{'driver_statistics:avg_daily_trips': [203,
  467,
  271,
  529,
  588,
  94,
  10,
  680,
  831,
  827],
 'driver_id': [531808,
  441196,
  324207,
  983271,
  656379,
  570408,
  519947,
  144996,
  379319,
  616812]}

In [50]:
time.sleep(10)
pd.DataFrame(features)

,driver_statistics:avg_daily_trips,driver_id
0,203,531808
1,467,441196
2,271,324207
3,529,983271
4,588,656379
5,94,570408
6,10,519947
7,680,144996
8,831,379319
9,827,616812


The features can now be used as an input to the trained model.

## Ingestion from Streaming (real-time) Source

With a streaming source, we can use Feast SDK to launch a Spark streaming job that continuously update the online store. First, we will update `driver_trips` feature table such that a new streaming source is added.

In [51]:
!pip install confluent_kafka

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
import json
import pytz
import io
import avro.schema
from avro.io import BinaryEncoder, DatumWriter
from confluent_kafka import Producer

In [53]:
# Change this to any Kafka broker addresses which is accessible by the spark cluster
KAFKA_BROKER = os.getenv("DEMO_KAFKA_BROKERS", "kafka:9092")

In [54]:
avro_schema_json = json.dumps({
    "type": "record",
    "name": "DriverTrips",
    "fields": [
        {"name": "driver_id", "type": "long"},
        {"name": "trips_today", "type": "int"},
        {
            "name": "datetime",
            "type": {"type": "long", "logicalType": "timestamp-micros"},
        },
    ],
})

In [55]:
driver_trips.stream_source = KafkaSource(
    event_timestamp_column="datetime",
    created_timestamp_column="datetime",
    bootstrap_servers=KAFKA_BROKER,
    topic="driver_trips",
    message_format=AvroFormat(avro_schema_json)
)
client.apply(driver_trips)

Start the streaming job and send avro record to Kafka:

In [56]:
job = client.start_stream_to_online_ingestion(
    driver_trips
)

In [57]:
def send_avro_record_to_kafka(topic, record):
    value_schema = avro.schema.parse(avro_schema_json)
    writer = DatumWriter(value_schema)
    bytes_writer = io.BytesIO()
    encoder = BinaryEncoder(bytes_writer)
    writer.write(record, encoder)
    
    producer = Producer({
        "bootstrap.servers": KAFKA_BROKER,
    })
    producer.produce(topic=topic, value=bytes_writer.getvalue())
    producer.flush()

In [58]:
# Note: depending on the Kafka configuration you may need to create the Kafka topic first, like below:
#from confluent_kafka.admin import AdminClient, NewTopic
#admin = AdminClient({'bootstrap.servers': KAFKA_BROKER})
#new_topic = NewTopic('driver_trips', num_partitions=1, replication_factor=3)
#admin.create_topics(new_topic)

for record in trips_df.drop(columns=['created']).to_dict('record'):
    record["datetime"] = (
        record["datetime"].to_pydatetime().replace(tzinfo=pytz.utc)
    )

    send_avro_record_to_kafka(topic="driver_trips", record=record)

### Retrieving joined features from several feature tables

In [60]:
time.sleep(15)
entities_sample = np.random.choice(entities, 10, replace=False)
entities_sample = [{"driver_id": e} for e in entities_sample]
entities_sample
time.sleep(15)

In [61]:
time.sleep(15)
features = client.get_online_features(
    feature_refs=["driver_statistics:avg_daily_trips", "driver_trips:trips_today"],
    entity_rows=entities_sample).to_dict()

In [62]:
time.sleep(15)
pd.DataFrame(features)

,driver_trips:trips_today,driver_statistics:avg_daily_trips,driver_id
0,508,514,371628
1,717,933,729702
2,834,639,561252
3,766,94,570408
4,386,681,225197
5,369,479,894474
6,738,252,492886
7,457,722,36610
8,150,943,267263
9,370,745,705022


In [39]:
# This will stop the streaming job
job.cancel()

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
